In [ ]:
import bw2data as bd
import bw2regional as bwr
from pathlib import Path

In [ ]:
PROJECT_NAME = "Spain case study"

In [ ]:
bd.projects.set_current(PROJECT_NAME)

In [ ]:
ls data

In [ ]:
data_dir = Path.cwd().absolute() / "data"
assert data_dir.is_dir()

In [ ]:
bwr.geocollections['popdensity'] = {'filepath': str(data_dir / 'gpw_v4_population_density.tif')}

In [ ]:
CROPS = ['cereals', 'citrus', 'rice', 'potatoe']
PLACES = ['cities', 'countries', 'regions']

In [ ]:
for crop in CROPS:
    for place in PLACES:
        bwr.calculate_needed_intersections({meal: 1}, ("water stress",), f'{place}-water-stress - {crop}')

In [ ]:
if 'cities-water-stress - popdensity' not in bwr.extension_tables:
    bwr.raster_as_extension_table('cities-water-stress', 'popdensity', engine='rasterstats')
if 'regions-water-stress - popdensity' not in bwr.extension_tables:
    bwr.raster_as_extension_table('regions-water-stress', 'popdensity', engine='rasterstats')
if 'countries-water-stress - popdensity' not in bwr.extension_tables:
    bwr.raster_as_extension_table('countries-water-stress', 'popdensity', engine='rasterstats')    

In [ ]:
meal = bd.get_node(name="meal")
lemon = bd.get_node(name="lemon")
rice = bd.get_node(name="rice")
mushroom = bd.get_node(name="mushroom")
cheese = bd.get_node(name="cheese")

In [ ]:
lca = bwr.ExtensionTablesLCA(
    demand={meal: 1},
    method=("water stress",),
    xtable='regions-water-stress - rice',
    limitations={
        'activities': [rice.id],
        'mode': 'include'
    },    
)
lca.lci()
lca.lcia()

In [ ]:
lca.score

In [ ]:
lca.geodataframe_xtable_spatial_scale().explore(column='score_rel')

In [ ]:
configuration = [
    {
        'xt': 'regions-water-stress - rice',
        'activities': [rice.id],
        'mode': 'include'
    },
    {
        'xt': 'regions-water-stress - citrus',
        'activities': [lemon.id],
        'mode': 'include'
    },
    {
        'xt': 'cities-water-stress - popdensity',
        'activities': [meal.id],
        'mode': 'include'
    },
    {
        'xt': 'countries-water-stress - popdensity',
        'activities': [mushroom.id, cheese.id],
        'mode': 'include'
    },
]

In [ ]:
matrix = None

In [ ]:
for conf in configuration:
    lca = bwr.ExtensionTablesLCA(
        demand={meal: 1},
        method=("water stress",),
        xtable=conf['xt'],
        limitations={
            'activities': conf['activities'],
            'mode': conf['mode']
        },    
    )
    lca.lci()
    lca.lcia()
    
    if matrix is None:
        matrix = lca.characterized_inventory
    else:
        matrix += lca.characterized_inventory

In [ ]:
matrix.todense()